In [61]:
import os
import operator
from collections import Counter

File to read titles from:

In [94]:
datafile = "/home/ctroupin/ULg/gher-ulg.github.io/Liege-Colloquium/wordle.dat"
datafile[:-1]

'/home/ctroupin/ULg/gher-ulg.github.io/Liege-Colloquium/wordle.da'

In [101]:
po = ["hows", "mot"]
pp2 = [x[:-1] if (x[-1] == "s") else x for x in po]
pp2

['how', 'mot']

In [105]:
with open(datafile, "r") as f:
    wordlist = f.readlines()

# Convert to lower case
wordlist = [x.lower() for x in wordlist]
# Remove final 's' (assuming it's plural)
wordlist = [x.rstrip()[:-1] if (x.rstrip()[-1] == "s") else x for x in wordlist]
# Convert list to string
wordstring = "".join(wordlist)
wordstring

" long-term studies in oceanography - a celebration of 50 years of science at the liege colloquium\n marine turbulence re³-visited\n submesoscale processes: mechanisms, implications and new frontier marine environmental monitoring, modelling and prediction\n low oxygen environments in marine, estuarine and fresh water primary production in the ocean: from the synoptic to the global scale\n remote sensing of colour, temperature and salinity – new challenges and opportunitie tracers of physical and biogeochemical processes, past changes and ongoing anthropogenic impact multiparametric observation and analysis of the sea\n science-based management of the coastal water influence of climate change on the changing arctic and subarctic condition turbulence re-revisited\n revisiting the role of zooplankton in pelagic ecosystem gas transfer at water surface marine environmental monitoring and prediction dying and dead sea tracer methods in geophysical fluid dynamic the use of data assimilation 

## Remove punctuation and common words

In [90]:
removedict = {"\n": " ", " the ": " ", " in ": " ", " a ": " ", 
              " of ": " ", " and ": " ", 
              " at ": " ", "/": " ", ", ": " ", 
              " : ": " ", ". ": "",
              "modeling": "modelling", "waters": "water", "seas": "sea",
              "environments": "environment"
               }
for oldvalue, newvalue in removedict.items():
    wordstring = wordstring.replace(oldvalue, newvalue)

## Count word frequency and sort

In [91]:
worddict = Counter(lstring.split(" "))
sortedworddict = sorted(worddict.items(), key=operator.itemgetter(1), reverse=True)
sortedworddict

[('', 49),
 ('ocean', 11),
 ('marine', 10),
 ('turbulence', 8),
 ('modelling', 7),
 ('processes', 6),
 ('seas', 5),
 ('hydrodynamics', 5),
 ('sea', 4),
 ('science', 3),
 ('water', 3),
 ('scale', 3),
 ('coastal', 3),
 ('dynamics', 3),
 ('coupled', 3),
 ('models', 3),
 ('ice', 3),
 ('shelf', 3),
 ('new', 2),
 ('environmental', 2),
 ('monitoring', 2),
 ('estuarine', 2),
 ('synoptic', 2),
 ('global', 2),
 ('remote', 2),
 ('sensing', 2),
 ('physical', 2),
 ('influence', 2),
 ('change', 2),
 ('revisited', 2),
 ('ecosystems', 2),
 ('geophysical', 2),
 ('data', 2),
 ('assimilation', 2),
 ('chemical', 2),
 ('three', 2),
 ('dimensional', 2),
 ('hydrodynamical', 2),
 ('covered', 2),
 ('mesoscale', 2),
 ('interactions', 2),
 ('deep', 2),
 ('atmosphere', 2),
 ('small', 2),
 ('mixing', 2),
 ('ecohydrodynamics', 2),
 ('long', 1),
 ('term', 1),
 ('studies', 1),
 ('oceanography', 1),
 ('celebration', 1),
 ('50', 1),
 ('years', 1),
 ('liege', 1),
 ('colloquium', 1),
 ('re³', 1),
 ('visited', 1),
 ('subm

# Convert the dictionary to json